In [1]:
%matplotlib inline
import cv2
from theano import *
from keras import *
import numpy as np
import seaborn as snb
import os
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using Theano backend.


In [2]:
fobia_folder = "./trypophobia_images/"
normal_folder = "./normal_pictures/"
resolution_x = 128
resolution_y = 128
def load_folder(folder):
    
    images = []
    
    for file in os.listdir(folder):
        image = cv2.imread(os.path.join(folder, file))
        if(image is not None):
            image = cv2.resize(image, (resolution_x, resolution_y), interpolation=cv2.INTER_LINEAR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image)
        
    return np.array(images)

In [3]:
fobia_images = load_folder(fobia_folder) /255.
normal_images = load_folder(normal_folder) /255.

In [4]:
data_y = [] # 1,0 -> fobia | 0,1 -> normal
data_x = []
for img in fobia_images:
    data_x.append(img)
    data_y.append([1,0])
for img in normal_images:
    data_x.append(img)
    data_y.append([0,1])
data_x = np.array(data_x)
data_y = np.array(data_y)

X_train, X_test, Y_train, Y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=4)

In [28]:
model = models.Sequential()
model.add(layers.InputLayer(input_shape=(resolution_x,resolution_y,3)))
model.add(layers.Convolution2D(nb_filter=32, nb_col=7, nb_row=7, border_mode='full'))
model.add(layers.Activation(activation="relu"))
model.add(layers.MaxPooling2D(dim_ordering="th", pool_size=(3,3)))
model.add(layers.Dropout(0.1))
#model.add(layers.Convolution2D(nb_filter=8, nb_col=3, nb_row=3))
#model.add(layers.Activation(activation="relu"))
#model.add(layers.MaxPooling2D(dim_ordering="th", pool_size=(3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(16))
model.add(layers.Activation(activation="relu"))
model.add(layers.Dense(2))
model.add(layers.Activation(activation="softmax"))

In [29]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
history = model.fit(X_train, Y_train, nb_epoch=10, batch_size=8, validation_data=(X_test, Y_test))

Train on 424 samples, validate on 106 samples
Epoch 1/10
424/424 [==============================] - 87s - loss: 0.8392 - acc: 0.6014 - val_loss: 0.6988 - val_acc: 0.5849

In [148]:
Y_train.shape

(424, 2)

In [ ]:
weights = simple_conv.layers[0].W.get_value(borrow=True)

plt.rcParams['figure.figsize'] = (10,5)
fig, axn = plt.subplots(4, 8, sharex=True, sharey=True)
cbar_ax = fig.add_axes([.91, .3, .03, .4])
for i, ax in enumerate(axn.flat):
    sns.heatmap(weights[i][0], ax=ax,
                cbar=i == 0,
                vmin=0, vmax=1,
                cbar_ax=None if i else cbar_ax,
               xticklabels=True, yticklabels=True)
plt.show()



In [ ]:
np.random.seed(4*4)
sample_image = X_test[np.random.randint(0, X_test_len)]
sns.heatmap(sample_image[0])


In [ ]:


convout1_f = theano.function(simple_conv.inputs, [simple_conv.layers[1].output])
transformed_sample_images = np.array(convout1_f([sample_image]))[0][0]

plt.rcParams['figure.figsize'] = (20,10)
fig, axn = plt.subplots(4, 8, sharex=True, sharey=True)
cbar_ax = fig.add_axes([.91, .3, .03, .4])
for i, ax in enumerate(axn.flat):
    sns.heatmap(transformed_sample_images[i], ax=ax,
                cbar=i == 0,
                vmin=0, vmax=1,
                cbar_ax=None if i else cbar_ax,
               xticklabels=False, yticklabels=False)
plt.show()

